# Modeling lines on Copernicus' heliograph
## Imports and tools

In [1]:
# <api>
import json
import pandas as pd
import numpy as np
from matplotlib.pyplot import *
from lmfit import *
from lmfit.models import *
from math import *
from scipy.optimize import *
import urllib.request

# Modelling the equatorial line

In [2]:
# <api>
geolat=radians(53+46/60+36/3600) # Latitude of Olstzyn
#print("Latitude of Olsztyn (rad): ",geolat)
eps=radians(23+33/60)# Copernicus obliquity of the ecliptic

-              Stundenwinkel vs. Azimuth
    - Norden:  180°    vs.   0°
    - Osten (früh):  270°   vs.  90°
    - Süden (mittag):    0°  vs.  180°
    - Westen (abends):   90°  vs.  270°

In [3]:
# <api>
def sunalt(l,H, geolat):
    ra,dec=ecl2equ(l,0, eps)
    az,alt=equ2hor(H,dec,geolat)
    return(az,alt)

In [4]:
# <api>
#all angles in radians
def normalize(x):
    return(x)
def equ2ecl(rra, rdecl, robl):
    lon = 0.0
    if cos(rra) != 0.0:
        lon = atan2(sin(rra)*cos(robl)+tan(rdecl)*sin(robl), cos(rra))
    lat = asin(sin(rdecl)*cos(robl)-cos(rdecl)*sin(robl)*sin(rra))
    lon = normalize(lon)
    return(lon, lat)

def ecl2equ(rlon, rlat, robl):
    ra = 0.0
    if cos(rlon) != 0.0:
        ra = atan2(sin(rlon)*cos(robl)-tan(rlat)*sin(robl), cos(rlon))
    decl = asin(sin(rlat)*cos(robl)+cos(rlat)*sin(robl)*sin(rlon))
    ra = normalize(ra)
    return(ra, decl)

# Azimuth measured westward from the south.
def equ2hor(rH, rdecl,rgeo): #add 180 to azi if wish to reckon azi from the North, DONE
    azi = 0.0
    denomin = cos(rH)*sin(rgeo)-tan(rdecl)*cos(rgeo)
    if denomin != 0.0:
        azi = atan2(sin(rH), denomin)
    alt = asin(sin(rgeo)*sin(rdecl)+cos(rgeo)*cos(rdecl)*cos(rH))
    azi = normalize(azi+pi) # !!! +pi !!!
   
    return(azi, alt)

def hor2equ(razi, ralt, rgeo):
    H = 0.0
    denomin = cos(razi)*sin(rgeo)+tan(alt)*cos(rgeo)
    if denomin != 0.0:
        H = atan2(sin(razi), denomin)
    decl = asin(sin(rgeo)*sin(ralt)-cos(rgeo)*cos(ralt)*cos(razi))
    return(H, decl)

from ephem import Equatorial, Ecliptic
p = Equatorial('90', '0', epoch="1517/3/11")
eps = float(Ecliptic(p).lat)

# test für sonne morgens und declination=0, equinox
az,alt=sunalt(0, 1.5*pi, geolat)
#print("azimuth (morgens expected -> 90°)=",degrees(az),"alt=",degrees(alt))

# test für sonne mittags und declination=0, equinox
az,alt=sunalt(0, 0, geolat)
#print("azimuth (mittags expected -> 180°)=",degrees(az),"alt=",degrees(alt))

# test für sonne abends und declination=0, equinox
az,alt=sunalt(0, pi/2, geolat)
#print("azimuth (abends expected -> 270°)=",degrees(az),"alt=",degrees(alt))

In [5]:
# <api>
def wall_x(d,az,nu): # all angles in radians, d in meter
    x=-d*sin(az)/sin(az-nu)
    return(x)
def wall_y(d,az,alt,nu):
    gamma=az-nu
    e=d*sin(pi-nu)/sin(gamma)
    y=e*tan(alt)
    return(y)

In [6]:
# <api>
def x_from_a(a):
    g=wall_x(dMirror,a,nu)
    return(g)
def a_from_x(xnull):
    azfromwall=lambda a: x_from_a(a)-xnull
    x=fsolve(azfromwall,2)+pi #+pi Korrektur
    w=x[0]
    return(w)

In [7]:
# <api>
# faz is an auxiliary function to compute its root for finding the hour angle H of the sun for a given azimuth with findHSun
def faz(H, azX, lsun):
    az,alt=sunalt(lsun, H, geolat)
    return(az-azX)

def faz1(lsun, azX, H):
    az,alt=sunalt(lsun, H, geolat)
    return(az-azX)

def findHSun(azX):
    x=fsolve(faz,2, args=(azX, 0))
    w=x[0]
    return(w)

In [8]:
# <api>
x=-2.423
lsun =0.0
dMirror=4.05
nu=2.13
azX=a_from_x(x) # corresponding azimuth of sun
H=findHSun(azX)
az,alt=sunalt(lsun,H, geolat)
y=wall_y(dMirror,az,alt,nu)
#print('azX=',azX,'az=',az,'H=',H,'y=',y)

In [9]:
xx=np.linspace(-10,3,50)
yy=[]
for i in xx:
    yy.append(a_from_x(i))

## Plot function

In [10]:
# <api>
# Define Plot Function
def Plot(x,y,report, headtitle):
    figure(figsize=(25,8))
    axis('equal')
    xlabel('x [m]', fontsize=18)
    ylabel('y [m]', fontsize=18)
    title(headtitle, fontsize=20)
   
    urllib.request.urlretrieve("http://repository.edition-topoi.org/CitableHandler/COPS/file/00022/Wand2Ortho_hell2_50perc.jpg?getDigitalFormat", "Wall.jpg")
    img=imread("Wall.jpg")
    
    scatter(x, y,zorder=1, color='red')
    f = imshow(img,zorder=0, extent=[-4.910895, 6.8745-4.910895, -0.478937, 1.5127-0.478937])
    
    #fit report
    dfit=list(report[0].keys())
    dcon=list(report[2].keys())
    v0=report[3]
    v1=[]
    v2=[]
    raw2=[]
    
    for j in range(len(dcon)):
        val = r'$%s$' %(str(dcon[j]))
        unit = ""
        middle=r'$%s$' %(str(round(report[2][dcon[j]], 3)))
        if dcon[j] == 'nu':
            val = r'$\mathrm{\nu}$'
            #unit = r'$ ^{\circ}$'
        if dcon[j] == 'geolat':
            val = r'$\mathrm{L_{Olsztyn}}$'
            middle=r'$%s$' %(str(round(degrees(report[2][dcon[j]]), 3)))
            unit = r'$ ^{\circ}$'
        if dcon[j] == 'x0':
            val = r'$\mathrm{x_0}$'
            unit = r'$\mathrm{m}$'
        if dcon[j] == 'y0':
            val = r'$\mathrm{y_0}$'
            unit = r'$\mathrm{m}$'
        if dcon[j] == 'dMirror':
            val = r'$\mathrm{d_{Mirror}}$'
            unit = r'$\mathrm{m}$'
    
        v1.append((val, "".join((middle, unit))))
       
    
    for j in range(len(dfit)):
        val = r'$\mathrm{%s}$' %(str(dfit[j]))
        rawval = str(dfit[j])
        unit = ""
        standerr=r'$\mathrm{%s}$' %(str(round(report[0][dfit[j]].stderr, 3)))
        middle=r'$\mathrm{%s}$' %(str(round(report[0][dfit[j]].value, 3)))
        rawmiddle = report[0][dfit[j]].value
        if dfit[j] == 'nu':
            val = r'$\nu$'
            #unit = r'$ ^{\circ}$'
        if dfit[j] == 'x0':
            val = r'$\mathrm{x_0}$'
            unit = r'$\mathrm{m}$'
        if dfit[j] == 'geolat':
            val = r'$\mathrm{L_{Olsztyn}}$'
            unit = r'$ ^{\circ}$' #dd
            middle=r'$%s$' %(str(round(degrees(report[0][dcon[j]]), 3)))
        if dfit[j] == 'y0':
            val = r'$\mathrm{y_0}$'
            unit = r'$\mathrm{m}$'
        if dfit[j] == 'dMirror':
            val = r'$\mathrm{d_{Mirror}}$'
            unit = r'$\mathrm{m}$'
      
        v2.append((val, "".join((middle, unit)), standerr))
        raw2.append((rawval, rawmiddle))
    
    chi2= r'$%s$' %(str(round(degrees(report[1]), 3)))
    v2.append((r'$\mathrm{\chi^2}$', chi2))
    df0=pd.DataFrame(v0)
    pd.set_option('max_colwidth',140)
    df0 = df0.rename(columns={0: 'data parameter(s)', 1: 'value(s)'})
    df1=pd.DataFrame(v1)
    df1 = df1.rename(columns={0: 'input parameter(s)', 1: 'value(s)'})
    df2=pd.DataFrame(v2)
    df2 = df2.rename(columns={0: 'fit parameters(s)', 1: 'value(s)', 2: 'standard error'})
    dffit = pd.concat([df1, df0, df2], axis=1)
    dffit=dffit.fillna('-')
    dfraw = pd.DataFrame(raw2)
    dfraw = dfraw.rename(columns={0: 'input parameter(s)', 1: 'value(s)'})
    return f, dffit, dfraw;   


## Daten

In [11]:
# <api>
messdaten = {0:[
    -4.866494068905198,
    0.1365968630305267,
    'None',
    -48,
    'horizontal',
    'h1'],
   1:[
    -4.810810212427215,
    0.12283128770872836,
    'None',
    -48,
    'horizontal',
    'h1'],
   2:[
    -4.101356633596621,
    -0.021782888199394224,
    'None',
    -48,
    'horizontal',
    'h1'],
   3:[
    -3.9989258358778628,
    -0.04401958679614537,
    'None',
    -48,
    'horizontal',
    'h1'],
   4:[
    -3.9198685087794924,
    -0.060961833346050986,
    'None',
    -48,
    'horizontal',
    'h1'],
   5:[
    -3.8311868114256677,
    -0.07926551042228819,
    'None',
    -48,
    'horizontal',
    'h1'],
   6:[
    -3.764503674655738,
    -0.09227473545168013,
    'None',
    -48,
    'horizontal',
    'h1'],
   7:[
    -3.7053825430865217,
    -0.10528396048107186,
    'None',
    -48,
    'horizontal',
    'h1'],
    8:[
    -4.858244608686237,
    0.34383684314990776,
    'None',
    -42,
    'horizontal',
    'h2'],
   9:[
    -4.771625276387153,
    0.3208437942607501,
    'None',
    -42,
    'horizontal',
    'h2'],
   10:[
    -3.982426915439942,
    0.14763957729966165,
    'None',
    -42,
    'horizontal',
    'h2'],
   11:[
    -3.8868706679036507,
    0.12570541881987313,
    'None',
    -42,
    'horizontal',
    'h2'],
   12:[
    -3.780315140075412,
    0.09938442864412694,
    'None',
    -42,
    'horizontal',
    'h2'],
   13:[
    -3.6696348821376934,
    0.07624010969648815,
    'None',
    -42,
    'horizontal',
    'h2'],
   14:[
    -3.528019148378873,
    0.04447339741541495,
    'None',
    -42,
    'horizontal',
    'h2'],
   15:[
    -3.432462900842582,
    0.02253923893562665,
    'None',
    -42,
    'horizontal',
    'h2'],
   16:[
    -3.2193518451861047,
    -0.024505749252057674,
    'None',
    -42,
    'horizontal',
    'h2'],
   17:[
    -3.0962973969199457,
    -0.05309579030502337,
    'None',
    -42,
    'horizontal',
    'h2'],
    18:[
    -4.869243888978185,
    0.5710444709888205,
    'None',
    -36,
    'horizontal',
    'h3'],
   19:[
    -4.775062551478387,
    0.5442696706376302,
    'None',
    -36,
    'horizontal',
    'h3'],
   20:[
    -3.7651911296739846,
    0.3069269488804706,
    'None',
    -36,
    'horizontal',
    'h3'],
   21:[
    -3.5905775550393226,
    0.2629073618624122,
    'None',
    -36,
    'horizontal',
    'h3'],
   22:[ -3.406339610149207, 0.220854285604611, 'None', -48, 'horizontal', 'h1'],
   23:[ -3.233788400569286, 0.180616450048585, 'None', -48, 'horizontal', 'h1'],
   24:[
    -2.239040989166311,
    -0.050978009486285336,
    'None',
    -36,
    'horizontal',
    'h3'],
   25:[
    -2.137985101484046,
    -0.07397105837544288,
    'None',
    -36,
    'horizontal',
    'h3'],
   26:[
    -2.0286797535828205,
    -0.09893061802485731,
    'None',
    -36,
    'horizontal',
    'h3'],
   27:[
    -1.9688711669953576,
    -0.11224238317121182,
    'None',
    -36,
    'horizontal',
    'h3'],
    28:[
    -4.874743529124158,
    0.7940165371902568,
    'None',
    -30,
    'horizontal',
    'h4'],
   29:[
    -4.759251086058713,
    0.7628549051431088,
    'None',
    -30,
    'horizontal',
    'h4'],
   30:[
    -3.7658785846922314,
    0.5055445356664177,
    'None',
    -30,
    'horizontal',
    'h4'],
   31:[
    -3.5617044442729613,
    0.4522974750809999,
    'None',
    -30,
    'horizontal',
    'h4'],
   32:[
    -3.3286571930873303,
    0.4013194653728018,
    'None',
    -30,
    'horizontal',
    'h4'],
   33:[
    -3.1127963173578665,
    0.3414165222142069,
    'None',
    -30,
    'horizontal',
    'h4'],
   34:[
    -2.2802882902611126,
    0.12404144817657886,
    'None',
    -30,
    'horizontal',
    'h4'],
   35:[
    -1.9289987759370488,
    0.03085909215209809,
    'None',
    -30,
    'horizontal',
    'h4'],
   36:[
    -1.4649666386205262,
    -0.08455996246913394,
    'None',
    -30,
    'horizontal',
    'h4'],
   37:[
    -1.369410391084235,
    -0.10452761018866541,
    'None',
    -30,
    'horizontal',
    'h4'],
  38:[
    -4.8444955083213035,
    0.9850706210521385,
    'None',
    -24,
    'horizontal',
    'h5'],
   39:[
    -4.7578761760222195,
    0.9575394704085418,
    'None',
    -24,
    'horizontal',
    'h5'],
   40:[
    -3.6400743163530853,
    0.6506125217499843,
    'None',
    -24,
    'horizontal',
    'h5'],
   41:[
    -3.3898406897112867,
    0.5817846451409927,
    'None',
    -24,
    'horizontal',
    'h5'],
   42:[
    -3.2269138503868184,
    0.5406391892340792,
    'None',
    -24,
    'horizontal',
    'h5'],
   43:[ -2.22872916389261, 0.2678992737922951, 'None', -24, 'horizontal', 'h5'],
 
   44:[
    -1.8313801633460174,
    0.16261531308931026,
    'None',
    -24,
    'horizontal',
    'h5'],
   45:[
    -1.5997078221968795,
    0.1007458591704588,
    'None',
    -24,
    'horizontal',
    'h5'],
   46:[
    -1.4092827821425433,
    0.050221659637704585,
    'None',
    -24,
    'horizontal',
    'h5'],
   47:[
    -3.681321617447887,
    0.9042924098231244,
    'None',
    -18,
    'horizontal',
    'h6'],
   48:[
    -3.3582177588719384,
    0.8079333825705362,
    'None',
    -18,
    'horizontal',
    'h6'],
   49:[
    -2.231478983965597,
    0.4681808312215365,
    'None',
    -18,
    'horizontal',
    'h6'],
   50:[
    -2.0190553833273666,
    0.4055550270102782,
    'None',
    -18,
    'horizontal',
    'h6'],
   51:[
    -1.8018195975614093,
    0.3405089018633192,
    'None',
    -18,
    'horizontal',
    'h6'],
   52:[
    -1.6746404191857698,
    0.30677567882198925,
    'None',
    -18,
    'horizontal',
    'h6'],
   53:[
    -0.5527138294071552,
    -0.028136230655608996,
    'None',
    -18,
    'horizontal',
    'h6'],
   54:[
    -0.415222825757815,
    -0.06671009556834051,
    'None',
    -18,
    'horizontal',
    'h6'],
   55:[
    -0.2873561923639292,
    -0.10770428141677268,
    'None',
    -18,
    'horizontal',
    'h6'],
   56:[
    -2.2486653594217647,
    0.6947833788265239,
    'None',
    -12,
    'horizontal',
    'h7'],
   57:[
    -2.0816137899878164,
    0.6425952086504754,
    'None',
    -12,
    'horizontal',
    'h7'],
   58:[
    -1.9551220666304236,
    0.5984243515739358,
    'None',
    -12,
    'horizontal',
    'h7'],
   59:[
    -1.8251930681817972,
    0.5541022244389149,
    'None',
    -12,
    'horizontal',
    'h7'],
   60:[ -1.715887720280572, 0.52097408163151, 'None', -12, 'horizontal', 'h7'],
   61:[
    -0.3581640592433395,
    0.0798705911000393,
    'None',
    -12,
    'horizontal',
    'h7'],
   62:[
    -0.21586087046627256,
    0.03085909215209809,
    'None',
    -12,
    'horizontal',
    'h7'],
   63:[
    -0.08455696198115259,
    -0.006958422468226866,
    'None',
    -12,
    'horizontal',
    'h7'],
   64:[
    0.06530823199662805,
    -0.054457220831355235,
    'None',
    -12,
    'horizontal',
    'h7'],
   65:[
    0.19386232040876106,
    -0.0939387060949744,
    'None',
    -12,
    'horizontal',
    'h7'],
   66:[
    -2.2699764649874123,
    0.9350002316948283,
    'None',
    -6,
    'horizontal',
    'h8'],
   67:[
    -2.1207987260278784,
    0.8782739597643408,
    'None',
    -6,
    'horizontal',
    'h8'],
   68:[
    -1.9619966168128906,
    0.8233629285356288,
    'None',
    -6,
    'horizontal',
    'h8'],
   69:[
    -1.834817438437251,
    0.7743514295876877,
    'None',
    -6,
    'horizontal',
    'h8'],
   70:[ -1.682202424386484, 0.719591668417457, 'None', -6, 'horizontal', 'h8'],
   71:[
    -0.359538969279833,
    0.2551925888798659,
    'None',
    -6,
    'horizontal',
    'h8'],
   72:[
    -0.130616448203682,
    0.1729016770660387,
    'None',
    -6,
    'horizontal',
    'h8'],
   73:[
    0.049496766576953455,
    0.10830936209452374,
    'None',
    -6,
    'horizontal',
    'h8'],
   74:[
    0.21036124084668195,
    0.04961657940377934,
    'None',
    -6,
    'horizontal',
    'h8'],
   75:[
    0.42140993144841854,
    -0.021026537906987774,
    'None',
    -6,
    'horizontal',
    'h8'],
   76:[
    -0.4626572220168379,
    0.5120491481811134,
    'None',
    0,
    'horizontal',
    'h9'],
   77:[
    -0.3024802027653566,
    0.4498771541452992,
    'None',
    -0,
    'horizontal',
    'h9'],
   78:[
    -0.1313039032219283,
    0.3869488098170782,
    'None',
    -0,
    'horizontal',
    'h9'],
   79:[
    -0.0013749047733027098,
    0.3424754126235761,
    'None',
    -0,
    'horizontal',
    'h9'],
   80:[
    0.21998561110213544,
    0.25927688045886116,
    'None',
    -0,
    'horizontal',
    'h9'],
   81:[
    0.3272285939486199,
    0.2181314245519475,
    'None',
    0,
    'horizontal',
    'h9'],
   82:[
    0.4097231961382244,
    0.1887850332065753,
    'None',
    0,
    'horizontal',
    'h9'],
   83:[
    0.4750314228716608,
    0.16775849507767454,
    'None',
    0,
    'horizontal',
    'h9'],
   84:[
    0.5142163589117228,
    0.15353910958043232,
    'None',
    0,
    'horizontal',
    'h9'],
   85:[
    -0.6173346011223453,
    0.762401094967665,
    'None',
    5,
    'horizontal',
    'h10'],
   86:[
    -0.49977979300215924,
    0.7185327780080881,
    'None',
    5,
    'horizontal',
    'h10'],
   87:[
    -0.3884120800461943,
    0.6746644610485111,
    'None',
    5,
    'horizontal',
    'h10'],
   88:[
    -0.18492539464517055,
    0.5955502204627912,
    'None',
    5,
    'horizontal',
    'h10'],
   89:[
    -0.02268601033894946,
    0.5335294964854582,
    'None',
    5,
    'horizontal',
    'h10'],
   90:[
    0.14986519924097141,
    0.4653067001103918,
    'None',
    5,
    'horizontal',
    'h10'],
   91:[
    0.31691676867491925,
    0.40570629706875944,
    'None',
    5,
    'horizontal',
    'h10'],
   92:[
    0.49359270836432145,
    0.34005509168787534,
    'None',
    5,
    'horizontal',
    'h10'],
   93:[
    0.6310837120136616,
    0.2877156514533455,
    'None',
    5,
    'horizontal',
    'h10'],
   94:[
    0.7692621706812481,
    0.23537621121881563,
    'None',
    5,
    'horizontal',
    'h10'],
   95:[
    -0.5685252948268298,
    0.9481607267827012,
    'None',
    10,
    'horizontal',
    'h11'],
   96:[
    -0.48328087256423924,
    0.9129148031565584,
    'None',
    10,
    'horizontal',
    'h11'],
   97:[
    -0.3911619001191813,
    0.8767612591795277,
    'None',
    10,
    'horizontal',
    'h11'],
   98:[
    0.06462077697838087,
    0.6965986195282994,
    'None',
    10,
    'horizontal',
    'h11'],
   99:[
    0.19661214048174802,
    0.6427464787089565,
    'None',
    10,
    'horizontal',
    'h11'],
   100:[
    0.38841209057257675,
    0.5663550991759001,
    'None',
    10,
    'horizontal',
    'h11'],
   101:[
    0.5080292637475026,
    0.5193101109882157,
    'None',
    10,
    'horizontal',
    'h11'],
   102:[
    0.6303962569954145,
    0.47135750244964353,
    'None',
    10,
    'horizontal',
    'h11'],
   103:[
    0.701891578893072,
    0.4459441326247852,
    'None',
    10,
    'horizontal',
    'h11'],
   104:[
    0.1340537338212986,
    0.8867450830392936,
    'None',
    15,
    'horizontal',
    'h12'],
   105:[
    0.22617270626635566,
    0.8533144001149264,
    'None',
    15,
    'horizontal',
    'h12'],
   106:[
    0.3547267946784878,
    0.8020338502897655,
    'None',
    15,
    'horizontal',
    'h12'],
   107:[
    0.4392837619228329,
    0.7693595176578047,
    'None',
    15,
    'horizontal',
    'h12'],
   108:[
    0.5279654592766567,
    0.7351724844410308,
    'None',
    15,
    'horizontal',
    'h12'],
   109:[
    0.6310837120136616,
    0.6962960794113369,
    'None',
    15,
    'horizontal',
    'h12'],
   110:[
    0.7197654093674855,
    0.6605963456097501,
    'None',
    15,
    'horizontal',
    'h12'],
   111:[ 0.8111969267942971, 0.624745341749682, 'None', 15, 'horizontal', 'h12'],
   112:[
    0.8971288040751348,
    0.5922222791762024,
    'None',
    15,
    'horizontal',
    'h12'],
   113:[
    0.9658743058998054,
    0.5625733477138677,
    'None',
    15,
    'horizontal',
    'h12'],
   114:[
    0.36572607497043563,
    0.9882472922802459,
    'None',
    20,
    'horizontal',
    'h13'],
   115:[
    0.44409594705055966,
    0.9578420105255046,
    'None',
    20,
    'horizontal',
    'h13'],
  116:[
    0.5238407291671772,
    0.9268316485368382,
    'None',
    20,
    'horizontal',
    'h13'],
  117:[
    0.6008356912108068,
    0.8956700164896902,
    'None',
    20,
    'horizontal',
    'h13'],
  118:[
    0.6888299335463852,
    0.8608779030389913,
    'None',
    20,
    'horizontal',
    'h13'],
  119:[
    0.7857610911191699,
    0.8248756291204418,
    'None',
    20,
    'horizontal',
    'h13'],
   120:[ 0.877880063564227, 0.7888733552018923, 'None', 20, 'horizontal', 'h13'],
   121:[
    0.9425008352794171,
    0.7672417368390665,
    'None',
    20,
    'horizontal',
    'h13'],
   122:[
    0.9871854114654521,
    0.752871081283343,
    'None',
    20,
    'horizontal',
    'h13'],
   123:[
    0.7940105513381299,
    1.0153246327483985,
    'None',
    25,
    'horizontal',
    'h14'],
   124:[
    0.8215087520679978,
    1.0038281083038199,
    'None',
    25,
    'horizontal',
    'h14'],
   125:[
    0.8551940479620868,
    0.9912726934498719,
    'None',
    25,
    'horizontal',
    'h14'],
   126:[ 0.889566798874422, 0.9781121983619987, 'None', 25, 'horizontal', 'h14'],
   127:[
    0.9218771847320157,
    0.9660105936834946,
    'None',
    25,
    'horizontal',
    'h14'],
   128:[
    -3.5287066033971195,
    0.0450784776493402,
    -0.12,
    -35,
    'intersection',
    'v1'],
   129:[
    -3.667572517082953,
    0.28090849882168695,
    -0.12,
    -30,
    'intersection',
    'v1'],
   130:[
    -3.8050635207322934,
    0.5217304319239167,
    -0.12,
    -25,
    'intersection',
    'v1'],
   131:[
    -2.231478983965597,
    0.1107296830302244,
    0.14,
    -25,
    'intersection',
    'v2'],
   132:[
    -2.1826696776700816,
    0.25277226794416524,
    0.14,
    -20,
    'intersection',
    'v2'],
   133:[
    -2.121486181046125,
    0.4368679291159074,
    0.14,
    -15,
    'intersection',
    'v2'],
   134:[
    -2.0582403193674286,
    0.6356367859603356,
    0.14,
    -10,
    'intersection',
    'v2'],
   135:[
    -1.993619547652239,
    0.8406077152024971,
    0.14,
    -5,
    'intersection',
    'v2'],
  136:[
    -0.5252156286772873,
    0.729121682101779,
    0.40,
    5,
    'intersection',
    'v3'],
   137:[
    -0.4399712064146959,
    0.8971827170745033,
    0.40,
    10,
    'intersection',
    'v3'],
   138:[
    0.2797941976895979,
    -0.118898265744389,
    0.66,
    -12,
    'intersection',
    'v4'],
   139:[
    0.3512895195872545,
    0.007109692970533876,
    0.66,
    -6,
    'intersection',
    'v4'],
   140:[
    0.45234540726951966,
    0.177591048878959,
    0.66,
    0,
    'intersection',
    'v4'],
   141:[
    0.5355274644773704,
    0.32492808583974536,
    0.66,
    5,
    'intersection',
    'v4'],
   142:[
    0.6262715268859358,
    0.47408036350230703,
    0.66,
    10,
    'intersection',
    'v4'],
   143:[
    0.7307646896594333,
    0.6551506235044233,
    0.66,
    15,
    'intersection',
    'v4'],
   144:[
    0.8194463870132571,
    0.807782112512055,
    0.66,
    20,
    'intersection',
    'v4'],
  145:[
    -3.45996110157245,
    -0.08380361217672733,
    -0.12,
    'None',
    'vertical',
    'v1'],
   146:[
    -3.5575797141634817,
    0.09439251671424426,
    -0.12,
    'None',
    'vertical',
    'v1'],
   147:[
    -3.6077639304954903,
    0.18288550092580463,
    -0.12,
    'None',
    'vertical',
    'v1'],
   148:[
    -3.6682599721012004,
    0.28211865928953733,
    -0.12,
    'None',
    'vertical',
    'v1'],
   149:[
    -3.7266936486521693,
    0.37741879613275636,
    -0.12,
    'None',
    'vertical',
    'v1'],
   150:[
    -3.764503674655738,
    0.4468517529756731,
    -0.12,
    'None',
    'vertical',
    'v1'],
   151:[
    -3.8085007958235266,
    0.5202177313391037,
    -0.12,
    'None',
    'vertical',
    'v1'],
   152:[
    -3.839436271644628,
    0.5760363829187034,
    -0.12,
    'None',
    'vertical',
    'v1'],
   153:[
    -2.2741011950968923,
    -0.01754732656191793,
    0.14,
    'None',
    'vertical',
    'v2'],
   154:[
    -2.2514151794947512,
    0.051583090164036116,
    0.14,
    'None',
    'vertical',
    'v2'],
   155:[
    -2.217042428582416,
    0.1601949921536096,
    0.14,
    'None',
    'vertical',
    'v2'],
   156:[
    -2.192981502943782,
    0.23265335016615224,
    0.14,
    'None',
    'vertical',
    'v2'],
   157:[
    -2.137297646465799,
    0.3931508822148116,
    0.14,
    'None',
    'vertical',
    'v2'],
   158:[
    -2.1015499855169706,
    0.49495563157272654,
    0.14,
    'None',
    'vertical',
    'v2'],
   159:[
    -2.049303404130222,
    0.6624115863115256,
    0.14,
    'None',
    'vertical',
    'v2'],
   160:[
    -2.0238675684550937,
    0.7383491556691384,
    0.14,
    'None',
    'vertical',
    'v2'],
    161:[
    -2.0101184680901594,
    0.783125092979603,
    0.14,
    'None',
    'vertical',
    'v2'],
   162:[
    -1.94756006142971,
    0.9781121983619987,
    0.14,
    'None',
    'vertical',
    'v2'],
   163:[
    -0.5967109505749448,
    0.5814821050240302,
    0.40,
    'None',
    'vertical',
    'v3'],
   164:[
    -0.5685252948268298,
    0.6401748877147746,
    0.40,
    'None',
    'vertical',
    'v3'],
   165:[
    -0.5479016442794284,
    0.6819254238556134,
    0.40,
    'None',
    'vertical',
    'v3'],
   166:[
    -0.5238407186407938,
    0.7294242222187415,
    0.40,
    'None',
    'vertical',
    'v3'],
   167:[
    -0.5066543431846267,
    0.7648214159033657,
    0.40,
    'None',
    'vertical',
    'v3'],
   168:[
    -0.4805310524912523,
    0.8157994256115639,
    0.40,
    'None',
    'vertical',
    'v3'],
   169:[
    -0.4544077617978779,
    0.8695002963724254,
    0.40,
    'None',
    'vertical',
    'v3'],
   170:[
    -0.43515902128697004,
    0.9098894019869326,
    0.40,
    'None',
    'vertical',
    'v3'],
   171:[
    -0.41728519081255566,
    0.9402946837416737,
    0.40,
    'None',
    'vertical',
    'v3'],
  172:[
    0.2935432980545327,
    -0.0910645749838297,
    0.66,
    'None',
    'vertical',
    'v4'],
   173:[
    0.3189791337296599,
    -0.05143181966172916,
    0.66,
    'None',
    'vertical',
    'v4'],
   174:[
    0.36435116493394215,
    0.026169720339177693,
    0.66,
    'None',
    'vertical',
    'v4'],
   175:[
    0.4344715767951062,
    0.14688322700725515,
    0.66,
    'None',
    'vertical',
    'v4'],
   176:[
    0.47915615298114034,
    0.22977921905500748,
    0.66,
    'None',
    'vertical',
    'v4'],
   177:[
    0.5355274644773704,
    0.32341538525493224,
    0.66,
    'None',
    'vertical',
    'v4'],
   178:[
    0.5843367707728868,
    0.40449613660090905,
    0.66,
    'None',
    'vertical',
    'v4'],
   179:[
    0.6853926584551511,
    0.5757338428017409,
    0.66,
    'None',
    'vertical',
    'v4'],
   180:[ 0.800197646502351, 0.7738976194122438, 0.66, 'None', 'vertical', 'v4'],
   181:[
    0.8242585721409847,
    0.8118664040910502,
    0.66,
    'None',
    'vertical',
    'v4'], 
    182:[
    -3.5287066033971195,
    0.0450784776493402,
    -0.12,
    -35,
    'intersection',
    'v1h2'],
   198:[
    -3.667572517082953,
    0.28090849882168695,
    -0.12,
    -30,
    'intersection',
    'v1h3'],
   183:[
    -3.8050635207322934,
    0.5217304319239167,
    -0.12,
    -25,
    'intersection',
    'v1h4'],
   184:[
    -2.231478983965597,
    0.1107296830302244,
    0.14,
    -25,
    'intersection',
    'v2h4'],
   185:[
    -2.1826696776700816,
    0.25277226794416524,
    0.14,
    -20,
    'intersection',
    'v2h5'],
   186:[
    -2.121486181046125,
    0.4368679291159074,
    0.14,
    -15,
    'intersection',
    'v2h6'],
   187:[
    -2.0582403193674286,
    0.6356367859603356,
    0.14,
    -10,
    'intersection',
    'v2h7'],
   188:[
    -1.993619547652239,
    0.8406077152024971,
    0.14,
    -5,
    'intersection',
    'v2h8'],
  189:[
    -0.5252156286772873,
    0.729121682101779,
    0.40,
    5,
    'intersection',
    'v3h10'],
   190:[
    -0.4399712064146959,
    0.8971827170745033,
    0.40,
    10,
    'intersection',
    'v3h11'],
   191:[
    0.2797941976895979,
    -0.118898265744389,
    0.66,
    -12,
    'intersection',
    'v4h7'],
   192:[
    0.3512895195872545,
    0.007109692970533876,
    0.66,
    -6,
    'intersection',
    'v4h8'],
   193:[
    0.45234540726951966,
    0.177591048878959,
    0.66,
    0,
    'intersection',
    'v4h9'],
   194:[
    0.5355274644773704,
    0.32492808583974536,
    0.66,
    5,
    'intersection',
    'v4h10'],
   195:[
    0.6262715268859358,
    0.47408036350230703,
    0.66,
    10,
    'intersection',
    'v4h11'],
   196:[
    0.7307646896594333,
    0.6551506235044233,
    0.66,
    15,
    'intersection',
    'v4h12'],
   197:[
    0.8194463870132571,
    0.807782112512055,
    0.66,
    20,
    'intersection',
    'v4h13']
             }